In [1]:
# XGBOOST example > 90%

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
from pandas import nan

In [2]:
def label_marker(row):
    if isinstance(row['marker'], str) and 'Natural' in row['marker']:
        return 0
    if isinstance(row['marker'], str) and 'Attack' in row['marker']:
        return 1
    return 

In [3]:
#path = './data/Zero_Day/'
path = '../Autoencoders/new_data/'
#path_normal = path +'VAE_syntheticdata_normal_epoch100_1000000.csv'
path_normal = path +'normal.csv'
path_attack = path + 'attack.csv' # real data

normal_df = pd.read_csv(path_normal)  
attack_df = pd.read_csv(path_attack)  

normal_df['marker'] = normal_df.apply(label_marker, axis=1)
attack_df['marker'] = attack_df.apply(label_marker, axis=1)

normal_label = normal_df['marker'].to_numpy() 
attack_label = attack_df['marker'].to_numpy() 

print(len(normal_label))
print(len(attack_label))

22714
55663


In [4]:
input_df = pd.concat([normal_df, attack_df], ignore_index=True)
input_df = input_df.sample(frac=1) # we suffle the dataframe

input_df = input_df.replace([np.inf, -np.inf], 0)
#scaler = MinMaxScaler()
#input_df = scaler.fit_transform(input_df)


In [6]:
y_col = 'marker'
X_cols = input_df.loc[:, input_df.columns != y_col].columns

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_df[X_cols], input_df[y_col],test_size=0.2, 
                                                    random_state=42)

print(' X_train shape',X_train.shape, '\n', 
      'y_train shape', y_train.shape, '\n',
      'X_test shape',X_test.shape, '\n', 
      'y_test shape', y_test.shape)

 X_train shape (62701, 128) 
 y_train shape (62701,) 
 X_test shape (15676, 128) 
 y_test shape (15676,)


In [7]:
from collections import Counter
class_counter = Counter(y_train)
print(' Number of items class 0:', class_counter[0], '\n',
      'Number of items class 1:', class_counter[1])
estimate_imbalancing = class_counter[0]/class_counter[1]
print('Imbalancing Factor: ',estimate_imbalancing)


 Number of items class 0: 18108 
 Number of items class 1: 44593
Imbalancing Factor:  0.4060727019935864


In [8]:

regressor = xgb.XGBRegressor(
    n_estimators=3000,
    scale_pos_weight=estimate_imbalancing,
    reg_lambda=1,
    gamma=0,
    max_depth=6
)

regressor = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, enable_categorical=False,
             gpu_id=-1, importance_type=None,
             interaction_constraints='', 
             max_delta_step=0,
             monotone_constraints='()', n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             tree_method='exact',
             learning_rate=0.300000012,
             validate_parameters=1, verbosity=None,
             n_estimators=500,
             scale_pos_weight=estimate_imbalancing,
             reg_lambda=1,
             colsample_bytree=0.6,
             min_child_weight = 10,
             gamma=0.5,
             max_depth=5
)

regressor = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=1, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.30000012,
             max_delta_step=0, max_depth=7, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=123,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=0.3278456267435863,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [9]:
regressor.fit(X_train, y_train,verbose=1)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, enable_categorical=False,
             gamma=0.5, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=5, min_child_weight=10, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=0.4060727019935864, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
y_pred = regressor.predict(X_test)
mean_squared_error(y_test, y_pred)

0.13751862950459726

In [11]:
y_pred_round = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, y_pred_round)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.37%


In [ ]:
sns.pairplot(input_df.sample(10), diag_kind='kde', hue='marker')

In [ ]:
#EOF